In [ ]:
import pandas as pd
import numpy as np
import pymysql
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp

In [ ]:
# выгрузим свежий Сфинкс
def get_connection():
    connection = pymysql.connect(
        host='',
        port=
    )
    return connection

query = '''
select
nativeitemid,
title,
duration,
numberoscarawards,
numberemmyawards,
year,
categorynames,
countryname,
description,
agerating,
kinopoiskrating,
imdbrating,
genretitles,
rolenames,
personnames,
refbooktitles,
refbooktypes
from datamart
where type = 'MOVIE' and adult != 'ADULT'
LIMIT 500000 OPTION max_matches=500000;
'''
conn = get_connection()
df = pd.read_sql(sql=query, con=conn)
conn.close()
df['nativeitemid'] = df['nativeitemid'].astype(int)
df[:1]

In [ ]:
df[df['nativeitemid'] == 2100180]

### Подготовка таблиц


In [ ]:
df = df.fillna('')

In [ ]:
# разложим в столбцы refbooktitles
ref_dict = {
    1:   'Языки вещания',
    2:   'Ключевые слова',
    3:   'Сеттинги',
    4:   'Временные периоды',
    5:   'Настроения',
    6:   'Стили',
    7:   'Темы',
    8:   'Киностудия',
    9:   'Жанры EpgService',
    10:  'Сериалы EpgService',
    11:  'Категории EpgService'
}

meta_list = []
for string, ids in df[['refbooktitles', 'refbooktypes']].values[:]:
    meta = [''] * 11
    if string == '': # пропускаем пустые строки
        meta_list.append(meta)
        continue

    string = np.array(string.split(','))
    ids = np.array([int(i) for i in ids.split(',')])

    if len(string) != len(ids): # какая-то хрень, не сопрадают иногда длины 2 полей
        min_len = min(len(string), len(ids))
        string = string[:min_len]
        ids = ids[:min_len]

    for i in range(1, 12):
        meta[i - 1] = ','.join(string[ids == i]) # индексируем np.array
    meta_list.append(meta)

# соединяем все это дело
df = pd.concat((
    df,
    pd.DataFrame(meta_list, columns=ref_dict.values())
), axis=1)

In [ ]:
# разложим в столбцы personnames
person_types = [
    'Актёр',
    'Режиссёр',
    'Сценарист',
    'Ведущий',
    'Композитор',
    'Оператор',
    'Продюсер'
]

person_list = []
for string, ids in df[['personnames', 'rolenames']].values[:]:
    person = [''] * 7
    if string == '': # пропускаем пустые строки
        person_list.append(person)
        continue

    string = np.array(string.split(','))
    ids = np.array([s for s in ids.split(',')])

    if len(string) != len(ids): # какая-то хрень, не сопрадают иногда длины 2 полей
        min_len = min(len(string), len(ids))
        string = string[:min_len]
        ids = ids[:min_len]

    for i in range(len(person_types)):
        person[i] = ','.join(string[ids == person_types[i]]) # индексируем np.array
    person_list.append(person)

# соединяем все это дело
df = pd.concat((
    df,
    pd.DataFrame(person_list, columns=person_types)
), axis=1)

In [ ]:
# нормальный формат даты
df['year'] = pd.to_datetime(df['year'], unit='s').dt.year

In [ ]:
# распрасим описание на слова, приведенные к нормальной форме
morph = pymorphy2.MorphAnalyzer()
vectorizer = CountVectorizer(max_df=100, min_df=1)

stop_words = [
'и','в','во','не','что','он','на','я','с','со','как','а','то','все','она','так','его',
    'но','да','ты','к','у','же','вы','за','бы','по','только','ее','мне','было','вот','от','меня','еще',
    'нет','о','из','ему','теперь','когда','даже','ну','вдруг','ли','если','уже','или','ни','быть','был',
    'него','до','вас','нибудь','опять','уж','вам','ведь','там','потом','себя','ничего','ей','может','они',
    'тут','где','есть','надо','ней','для','мы','тебя','их','чем','была','сам','чтоб','без','будто','чего','раз',
    'тоже','себе','под','будет','ж','тогда','кто','этот','того','потому','этого','какой','совсем','ним','здесь',
    'этом','один','почти','мой','тем','чтобы','нее','сейчас','были','куда','зачем','всех','никогда','можно','при',
    'наконец','два','об','другой','хоть','после','над','больше','тот','через','эти','нас','про','всего','них','какая',
    'много','разве','три','эту','моя','впрочем','хорошо','свою','этой','перед','иногда','лучше','чуть','том','нельзя',
    'такой','им','более','всегда','конечно','всю','между'
]

def get_normolize_text(text):
    vectorizer.fit([text])
    features = []

    for f in vectorizer.get_feature_names():
        norm_f = morph.parse(f)[0].normal_form
        if norm_f not in stop_words:
            features.append(norm_f)


    return ','.join(features)

df['norm_description'] = df['description'].apply(get_normolize_text)

### Подготовка матриц для расстояний

In [ ]:
number_dict = dict(zip(
    df.index,
    df['nativeitemid']))

asset_dict = dict(zip(
    df['nativeitemid'],
    df.index))

name_dict = dict(zip(
    df['nativeitemid'],
    df['title']
))

In [ ]:
data_corpus = df['norm_description'].tolist()
vectorizer = CountVectorizer(max_df=500, min_df=10) # это параметры для тюнинга ключевых слов
X = vectorizer.fit_transform(data_corpus)

words = vectorizer.get_feature_names()
matrix = X.toarray()

def get_key_words(index):
    '''
    возвращает список слов, через "," которые прошли фильтр CountVectorizer для конкретного фильма
    '''
    key_words = [w for w, v in zip(words, matrix[index]) if v != 0]
    return key_words #','.join(key_words)

key_words = list(map(get_key_words, df.index))
df['key_words'] = list(map(lambda x: ','.join(x), key_words))

In [ ]:
def r():
    return round(np.random.uniform(-0.2, 0.2), 2)
#     return 0
r()

-0.01

In [ ]:
################# ТЮНИНГ ########################
features = [
#     ('categorynames', 0.7 + r()),
    ('countryname', 1.0 + r()),
    ('genretitles', 1.2 + r()),
    ('Ключевые слова', 0.5 + r()),
    ('Сеттинги', 0.5 + r()),
    ('Временные периоды', 0.5 + r()),
#     ('Настроения', 1 + r()),
    ('Стили', 0.5 + r()),
    ('Темы', 0.5 + r()),
    ('Киностудия', 1.5 + r()),
    ('Категории EpgService', 0.5 + r()),
    ('key_words', 1.5 + r()),
    # главные актеры
    ('Актёр', 1.5 + r()),
    ('Режиссёр', 1.5 + r()),
    ('Сценарист', 1.2 + r()),
#     ('Ведущий', 2 + r()),
    ('Композитор', 0.7 + r()),
    ('Оператор', 0.5 + r()),
    ('Продюсер', 0.2 + r())
]

In [ ]:
# удалим пробелы у персон
for f, _ in features:
    df[f] = df[f].apply(lambda x: x.replace(' ', ''))

In [ ]:
def get_matrix(feature, max_df=10000, min_df=1):
    '''описать'''
    data_corpus = df[feature]
    vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df)
    X = vectorizer.fit_transform(data_corpus).toarray()
    F = vectorizer.get_feature_names()
    return X, F

In [ ]:
group_matrix = [] # матрица, сложенная из групп фичей
feature_list = [] # список фич

for feature, _ in features:
#     print(feature)
    add_matrix, feature_names =  get_matrix(feature)
    group_matrix.append(add_matrix)

    feature_list.extend(feature_names)

In [ ]:
feature_list = np.array(feature_list)
feature_list = feature_list.reshape(1, feature_list.shape[0])
feature_list.shape

(1, 52397)

In [ ]:
# тут мы можем перебирать веса сколько угодно

# тут нужно складывать уже sparse matrix, чтобы это работало быстрее
matrix = np.array([[]] * df.shape[0])
for i, m in enumerate(group_matrix):
    matrix = np.concatenate(
    (
        matrix,
        m * features[i][1] # домнажаем матрицу на вес категории
    ),
    axis=1
)

matrix = sp.csr_matrix(matrix) # чтобы быстро считался cos

In [ ]:
# result_dict = {}

# for asset_index in range(df.shape[0]):
#     asset_id = number_dict[asset_index]

#     # считаем похожих
#     similarities = cosine_similarity(matrix, matrix[asset_index])
#     similarities = similarities.ravel() # 2d -> 1d
#     top = similarities.argsort()[-21:-1][::-1] # первые 20 похожих пользователей от самого релевантного

#     result_dict[asset_id] = [number_dict.get(i, 0) for i in top]

In [ ]:
def asset_feature(number):
    '''
    возвращает список фич ассета
    '''
    m = matrix[number].toarray()
    return set(feature_list[m != 0])

result = []

for asset_index in range(df.shape[0]):
    asset_id = number_dict[asset_index]

    # считаем похожих
    similarities = cosine_similarity(matrix, matrix[asset_index])
    similarities = similarities.ravel() # 2d -> 1d
    top = similarities.argsort()[-26:][::-1] # первые 20 похожих пользователей от самого релевантного

    for i in top:
        re_assetid = number_dict[i]
        r = [
                asset_id, # assetid
                re_assetid, # re_assetid
                similarities[i], # score
                name_dict[re_assetid], # title
                str(asset_feature(asset_index) & asset_feature(i)) # features
            ]

        result.append(r)

### Обработка 4 Т

In [ ]:
table_to_mysql = pd.DataFrame(result, columns=['assetid', 're_assetid', 'score', 'title', 'features'])

In [ ]:
table_to_mysql.shape

In [ ]:
suffix = [
    'UHD',
    'UHD HDR',
    '(Сурдоперевод)',
    '(версия с тифлокомментарием)'
]
def title_cut(title):
    i = max([title.find(suf) - 1 for suf in suffix])
    if i > 0:
        return title[:i]
    else:
        return title

table_to_mysql['title_cut'] = table_to_mysql['title'].apply(title_cut)

In [ ]:
table_to_mysql.sort_values(by=['assetid', 'score'], ascending=[False, False], inplace=True)

In [ ]:
table_to_mysql

In [ ]:
table_to_mysql.drop_duplicates(subset=['assetid', 'title_cut'], keep='first', inplace=True)
table_to_mysql.drop('title_cut', axis=1, inplace=True)

In [ ]:
table_to_mysql.shape

In [ ]:
table_to_mysql

In [ ]:
table_to_mysql = table_to_mysql[table_to_mysql['score'].round(2) != 1]

In [ ]:
table_to_mysql.to_csv('similar_meta.csv', index=False)

In [ ]:
# сюда скрипт по загрузке в базу

In [ ]:
table_to_mysql.shape[0] / 12000

In [ ]:
table_to_mysql